In [1]:
import sys

from pathlib import Path

from nipype.interfaces import utility as niu
from nipype.pipeline import engine as pe
from niworkflows.engine.workflows import LiterateWorkflow as Workflow

# Nodes
sys.path.append('/opt/pynordic')
from pynordic.interfaces.nordic import Nordic

from IPython.display import clear_output

In [2]:
def find_file(root_path, file_pattern):
    
    import glob
    from pathlib import Path
    
    file_paths  = list(Path(root_path).rglob(file_pattern))
    if len(file_paths) != 1:
        raise ValueError(f"Error: {len(file_paths)} paths were found. 1 path is expected.")
    
    for file_path in file_paths:
        return file_path
    
# DIR
bids_dir = '/data/1_attention/7T/bids'
sub_id, ses_id, task_id, run_id = '000', 'HeadPilot01', 'AttendAwayQ1', '01'
patch_size=[23,23,16]
n_threads = 4


# raw bold
bold_data_mag = find_file(bids_dir,f'*sub-{sub_id}_ses-{ses_id}_task-{task_id}_*_run-{run_id}_part-mag_bold.nii.gz')
bold_data_phase = find_file(bids_dir,f'*sub-{sub_id}_ses-{ses_id}_task-{task_id}_*_run-{run_id}_part-phase_bold.nii.gz')

for i in [
    bold_data_mag,
    bold_data_phase
]:
    print(i)
    if not Path(i).exists():
        raise ValueError(f"{i} does not exist.")

/data/1_attention/7T/bids/sub-000/ses-HeadPilot01/func/sub-000_ses-HeadPilot01_task-AttendAwayQ1_acq-mb4_dir-RL_run-01_part-mag_bold.nii.gz
/data/1_attention/7T/bids/sub-000/ses-HeadPilot01/func/sub-000_ses-HeadPilot01_task-AttendAwayQ1_acq-mb4_dir-RL_run-01_part-phase_bold.nii.gz


In [3]:
def _index_list(X,idx):
    return X[idx]

In [4]:
workflow = Workflow(
    name = 'nordic_example',
    base_dir = '/notebooks'
)

inputnode = pe.Node(
    niu.IdentityInterface(['mag_image','phase_image','patch_size','n_threads']),
    name='inputnode'
)
inputnode.inputs.mag_image = str(bold_data_mag)
inputnode.inputs.phase_image = str(bold_data_phase)
inputnode.inputs.patch_size = patch_size
inputnode.inputs.n_threads = n_threads



nordic_proc = pe.Node(
    Nordic(out_image='nordic_processed'),
    name='nordic_process'
)

workflow.connect([
    (inputnode,nordic_proc,[
        ('mag_image','mag_image'),
        ('phase_image','phase_image'),
        ('n_threads','n_threads'),
        (('patch_size',_index_list,0),'patch_x_dim'),
        (('patch_size',_index_list,1),'patch_y_dim'),
        (('patch_size',_index_list,2),'patch_z_dim'),
    ]),
])

workflow.run()

clear_output(wait=True)

230325-05:16:43,331 nipype.workflow INFO:
	 Workflow nordic_example settings: ['check', 'execution', 'logging', 'monitoring']
230325-05:16:43,335 nipype.workflow INFO:
	 Running serially.
230325-05:16:43,335 nipype.workflow INFO:
	 [Node] Setting-up "nordic_example.nordic_process" in "/notebooks/nordic_example/nordic_process".
230325-05:16:43,339 nipype.workflow INFO:
	 [Node] Executing "nordic_process" <pynordic.interfaces.nordic.Nordic>
230325-05:16:43,357 nipype.interface INFO:
	 stdout 2023-03-25T05:16:43.357795:------------------------------------------
230325-05:16:43,359 nipype.interface INFO:
	 stdout 2023-03-25T05:16:43.359277:Setting up environment variables
230325-05:16:43,360 nipype.interface INFO:
	 stdout 2023-03-25T05:16:43.359277:---
230325-05:16:43,361 nipype.interface INFO:
	 stdout 2023-03-25T05:16:43.359277:LD_LIBRARY_PATH is .:/opt/matlab/runtime/glnxa64:/opt/matlab/bin/glnxa64:/opt/matlab/sys/os/glnxa64:/opt/matlab/sys/opengl/lib/glnxa64
230325-05:16:45,505 nipype